In [ ]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [ ]:
from exp import nb_resnet_ssd_std

In [ ]:
import sys 
sys.path.append("..") 
#from exp import nb_bbox_hw_statistics
from detect_symbol.exp import nb_databunch
from detect_symbol.exp import nb_init_model
from detect_symbol.exp import nb_resnet_ssd
from detect_symbol.exp import nb_anchors_loss_metrics

In [ ]:
# export
import torch

In [ ]:
# export
from torch.nn import functional as F

In [ ]:
# export
from torch import tensor

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
# export
from IPython.core import debugger as idb

In [ ]:
# export
import numpy as np

In [ ]:
# export
import math

## functions

In [ ]:
def find_neibs(idx, grids = (49, 49), dis = 1, dbg = None):
    '''
    找到某个anchor周围相邻的anchors的下标里列表。距离默认1。
    这个任务中只有第一层的grids参与，所以只需要第一次的grids的尺寸。
    anchor也是1对1的。
    参数：
        idx：目标anchor在grid anchors(get_grid_anchors返回的gvs)列表中的下标
        grids: 尺寸
        dis：邻居的距离
    返回值：
        邻居的下标列表
    '''
    gh, gw = grids
    x = idx % gh
    y = idx // gw
    ret = []
    for nx in range(x - dis, x + dis + 1):
        for ny in range(y - dis, y + dis + 1):
            if nx >= 0 and ny >= 0 and nx < gw and ny < gh \
                    and not(nx == x and ny == y):
                nidx = ny * gw + nx
                if dbg:
                    print(x, y, nx, ny, nidx)
                ret += [nidx]
    return ret      
                
    

In [ ]:
# export
#定义一个新的GridAnchor_Functions，主要是修改get_scroe_hits
class GridAnchor_Funcs_std(nb_anchors_loss_metrics.GridAnchor_Funcs):
    def __init__(self, fig_hw, grids, device):
        anchors = [[(1, 1)]]
        gvs,ghs,gws,avs,ahs,aws = nb_anchors_loss_metrics.get_grids_anchors( \
                    fig_hw, grids, anchors)
        self.grids = grids
        super().__init__(gvs, avs, device)
        
    def get_scores_hits(self, gt_bboxs): 
        # ground truch bbox center x,y
        gt_cx = gt_bboxs[:,[0,2]].mean(-1)
        gt_cy = gt_bboxs[:,[1,3]].mean(-1)

        # 判断目标bbox的中心落在哪个cell内
        hits = ((gt_cx[:,None] >= self.gvs[:,0][None]) &
                (gt_cx[:,None] <  self.gvs[:,2][None]) &
                (gt_cy[:,None] >= self.gvs[:,1][None]) &
                (gt_cy[:,None] <  self.gvs[:,3][None]))
        
        # 在的就是1，不在就是0
        scores = hits * 1
        
        return scores,hits

In [ ]:
# export
def yolo_L_std(pred_batch, *gt_batch, conf_th=0.5,
           lambda_cent=1, lambda_pconf=1, lambda_nconf=1, lambda_clas=1, clas_weights=None, gaf):
    '''
    与detect_symbol里面的yolo_L相比的区别是：
        不计算宽高方面的损失
        neg_idx要去掉find_neibs返回的discard列表
        
    clas_weights: 
    为了解决数据集的imbalance问题，一种方法是在dataloader中使用WeightedRandomSampler，但是这种方法不适用于目标检测问题。
    因为，（1）目标检测的label不是一个简单的数值（2）目标检测问题的一张图片可能包括不同类别的多个目标。
    所以为了解决目标检测问题中的imbalance问题，我们的方法是在损失函数中使用权重。
    为各类别分配权重，各目标对应的损失乘以该目标所属类别的权重。
    默认为None，即不使用权重。
    若设置非None，则clas_weights应该是一个一维tensor，其长度等于数据集的类别数。
    若设置为全1，则相当于不使用权重。
    合理的设置应保证所有元素之和等于数据集的类别数，否则相当于对损失函数的整体做了缩放。
    '''
    clas_loss = 0
    cent_loss = 0
    pconf_loss = 0
    nconf_loss = 0
    pos_cnt = 0
    neg_cnt = 0
    
    for pred_txy,pred_conf,pred_clas,gt_bboxs,gt_clas in zip(*pred_batch, *gt_batch):
        keep = nb_anchors_loss_metrics.get_y(gt_bboxs)
        if keep.numel()==0: continue
          
        gt_bboxs = gt_bboxs[keep]
        gt_clas = gt_clas[keep]
        
        gt_bboxs = (gt_bboxs + 1) / 2
        gt_clas = gt_clas - 1 # the databunch add a 'background' class to classes[0], but we don't want that,so gt_clas-1
        
        if clas_weights is not None: ws = clas_weights[gt_clas]
        else: ws = None
        
        scores,hits = gaf.get_scores_hits(gt_bboxs)
        idx = nb_anchors_loss_metrics.idx_fromScoresHits(scores,hits)
        
        # classification loss
        pred_clas = pred_clas[idx]
        clas_loss += F.cross_entropy(pred_clas, gt_clas, weight=clas_weights, reduction='sum')
        
        # bbox center loss
        gt_t = gaf.b2t(gt_bboxs,idx,eps=1)
        pred_txy = pred_txy[idx]
        if ws is not None:
            cent_loss += ((gt_t[...,:2]-pred_txy)*ws[...,None]).abs().sum()
        else:
            cent_loss += (gt_t[...,:2]-pred_txy).abs().sum()
        
        # positive confidence loss
        conf_pos = pred_conf[idx]
        if ws is not None: 
            pconf_loss += F.binary_cross_entropy_with_logits(conf_pos,torch.ones_like(conf_pos),weight=ws[...,None],reduction='sum')
        else: 
            pconf_loss += F.binary_cross_entropy_with_logits(conf_pos,torch.ones_like(conf_pos),reduction='sum')
        #import pdb; pdb.set_trace()
        # negative conficence loss
        scores[range(0,len(idx)),idx] = conf_th + 1 # 强制责任anchor的得分好于threshold
        
        #这里把邻居也的conf也调高。后面就被过滤掉了
        #scores[range(0,len(discards)),discards] = conf_th + 1
                
        tmp = scores>conf_th # 判断各得分是否好于threshold
        tmp = tmp.max(dim=0)[0] # 各anchor是否有任意一个好于threshold的得分
        
        #取得命中的anchor周围的anchor的下标立标
        discards = []
        #print('tofind', idx)
        for hidx in idx:
            #gaf.grids:[(49,49)]
            #print('finding', hidx)
            neibs = find_neibs(hidx, gaf.grids[0], dis = 1)            
            for i in neibs:
                discards += [i]
        #print('discards:', discards)
        #import pdb; pdb.set_trace()        
        tmp[discards] = 1
         
        neg_idx = torch.where(tmp==0)[0] # 如果没有，该anchor是negative anchor
        
        conf_neg = pred_conf[neg_idx]
        nconf_loss += F.binary_cross_entropy_with_logits(conf_neg,torch.zeros_like(conf_neg),reduction='sum')
        
        pos_cnt += len(idx)
        neg_cnt += len(neg_idx)
        
    clas_loss  = lambda_clas  * clas_loss  /pos_cnt
    cent_loss  = lambda_cent  * cent_loss  /pos_cnt
    pconf_loss = lambda_pconf * pconf_loss /pos_cnt
    nconf_loss = lambda_nconf * nconf_loss /neg_cnt
    
    return clas_loss + cent_loss + pconf_loss + nconf_loss


## test

# export

In [ ]:
!python notebook2script.py --fname 'anchors_loss_metrics.ipynb' --outputDir './exp/'